Installing necessary libraries

In [ ]:
!pip install transformers torch tensorflow mlflow flask fastapi kafka-python prometheus_client opencensus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.5/637.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.

Importing libraries

In [ ]:
import transformers
import torch
import tensorflow as tf
import mlflow
from flask import Flask, request, jsonify
from fastapi import FastAPI
import kafka
from prometheus_client import start_http_server, Summary
import opencensus


Loading pre-trained LLM i.e GPT-2

In [ ]:
#import GPT-2 model for language modelling
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# initializing the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# initializing the model
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

just checking

In [ ]:
input_text = "Hello, how are you?"
tokens = tokenizer.encode(input_text)  # Converts text into token IDs
print(tokens)


[15496, 11, 703, 389, 345, 30]


just checking 2.0

In [ ]:
input_text = "The fat cat sat"
input_ids = tokenizer.encode(input_text, return_tensors="pt")  # Convert input text to token IDs
output = model.generate(input_ids, max_length=50)  # Generate text up to 50 tokens
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)  # Decode token IDs back to text
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The fat cat sat in the back seat of the car, and the fat cat sat in the front seat.

"I'm not going to lie, I'm not going to lie," he said. "I'm not going to lie.


Defining input and output - middleware


In [ ]:
# initiate Flask application
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate_text():
    input_text = request.json['input_text']
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=20, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    app.run() # <--
    # check this <--


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Creating a feedback loop - operational layer

In [ ]:
import mlflow

def track_performance_and_log_feedback(input_text, generated_text):
    mlflow.start_run()
    mlflow.log_param("model", "gpt2")
    mlflow.log_param("framework", "transformers")
    mlflow.log_param("input_text", input_text)
    mlflow.log_param("generated_text", generated_text)
    response_time = 1.2 # wait for the response time and then see to add the real value
    mlflow.log_metric("response_time", response_time)
    mlflow.end_run()


Set up monitoring. Using Prometheus. (ongoing)

In [ ]:
from prometheus_client import Summary, start_http_server

REQUEST_TIME = Summary('request_processing_seconds', 'Time spent processing request')

@app.route('/generate', methods=['POST'])
def generate_text():
    with REQUEST_TIME.time():
        input_text = request.json['input_text']
        inputs = tokenizer.encode(input_text, return_tensors='pt')
        outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        track_performance_and_log_feedback(input_text, generated_text)
        return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    start_http_server(8000)
    app.run()


ValueError: Duplicated timeseries in CollectorRegistry: {'request_processing_seconds', 'request_processing_seconds_sum', 'request_processing_seconds_created', 'request_processing_seconds_count'}

Deploy and testing (ongoing)

In [ ]:
import requests

response = requests.post('http://localhost:5000/generate', json={'input_text': 'Once upon a time'})
print(response.json())


ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7879fbf43510>: Failed to establish a new connection: [Errno 111] Connection refused'))